# 미니 배치와 데이터 로드
## (Mini Batch and Data Load)

## 데이터 로드하기(Data Load)
- 데이터셋(Dataset)
- 데이터로더(DataLoader)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.__version__

'1.9.1+cu111'

In [2]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [3]:
# 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(1)

In [4]:
# 데이터
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [5]:
# Build dataset
dataset = TensorDataset(x_train, y_train)

### dataset -> dataloader
- 데이터셋을 만들었다면 데이터로더를 사용 가능합니다. 

- 데이터로더는 기본적으로 2개의 인자를 입력받는다. 
    - 하나는 데이터셋 
    - 미니 배치의 크기입니다. 
    
- 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다.

In [6]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [7]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [12]:
model = MultivariateLinearRegressionModel()
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.5435,  0.3462, -0.1188]], requires_grad=True), Parameter containing:
tensor([0.2937], requires_grad=True)]


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [10]:
# Learning using the whole batch
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train) # forward()
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 31667.599609
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210559
Epoch 1100/2000 Cost: 0.209230
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206762
Epoch 1400/2000 Cost: 0.205618
Epoch 1500/2000 Cost: 0.204529
Epoch 1600/2000 Cost: 0.203481
Epoch 1700/2000 Cost: 0.202486
Epoch 1800/2000 Cost: 0.201539
Epoch 1900/2000 Cost: 0.200634
Epoch 2000/2000 Cost: 0.199770


In [11]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.2306]], grad_fn=<AddmmBackward>)


---
### Learning using mini-batch from dataloader
---

In [24]:
#  Initialize a model and an optimizer
model = MultivariateLinearRegressionModel()
print(list(model.parameters()))
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

[Parameter containing:
tensor([[ 0.1373,  0.4789, -0.2398]], requires_grad=True), Parameter containing:
tensor([-0.2437], requires_grad=True)]


In [25]:
# Learning using mini-batch from dataloader
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    print(batch_idx)
    print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

0
[tensor([[ 96.,  98., 100.],
        [ 73.,  66.,  70.]]), tensor([[196.],
        [142.]])]
Epoch    0/20 Batch 1/3 Cost: 19709.205078
1
[tensor([[73., 80., 75.],
        [93., 88., 93.]]), tensor([[152.],
        [185.]])]
Epoch    0/20 Batch 2/3 Cost: 6080.016602
2
[tensor([[89., 91., 90.]]), tensor([[180.]])]
Epoch    0/20 Batch 3/3 Cost: 2136.703125
0
[tensor([[73., 80., 75.],
        [93., 88., 93.]]), tensor([[152.],
        [185.]])]
Epoch    1/20 Batch 1/3 Cost: 581.063416
1
[tensor([[89., 91., 90.],
        [73., 66., 70.]]), tensor([[180.],
        [142.]])]
Epoch    1/20 Batch 2/3 Cost: 185.738266
2
[tensor([[ 96.,  98., 100.]]), tensor([[196.]])]
Epoch    1/20 Batch 3/3 Cost: 77.145729
0
[tensor([[ 96.,  98., 100.],
        [ 89.,  91.,  90.]]), tensor([[196.],
        [180.]])]
Epoch    2/20 Batch 1/3 Cost: 9.839544
1
[tensor([[73., 66., 70.],
        [93., 88., 93.]]), tensor([[142.],
        [185.]])]
Epoch    2/20 Batch 2/3 Cost: 26.419914
2
[tensor([[73., 80., 75.]]

In [26]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[153.7148]], grad_fn=<AddmmBackward>)


In [27]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.6858, 1.0082, 0.3098]], requires_grad=True), Parameter containing:
tensor([-0.2374], requires_grad=True)]
